# Read and dump calin and ACTL header and event

calin/examples/iact_data/read and dump calin and ACTL raw header and event from zfits file.ipynb - Stephen Fegan - 2017-03-09

Copyright 2017, Stephen Fegan <sfegan@llr.in2p3.fr>
LLR, Ecole polytechnique, CNRS/IN2P3, Universite Paris-Saclay

This file is part of "__calin__". "__calin__" is free software: you can redistribute it and/or modify it under the
terms of the GNU General Public License version 2 or later, as published by
the Free Software Foundation. "__calin__" is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.

## Introduction

__calin__ provides some level of access to the raw ACTL header and event protobuf structures. In Python it is possible to retrieve the header and any desired event from ZFits but __calin__ does not provide simple access to the fields in the protobuf in the usual way. However the messages can be serialized to JSON and then accessed through the Python JSON packages if desired. This is only really intended for debugging.

This example shows how both the __calin__ and __ACTL__ data structures can be retrieved at the same time.

In [ ]:
%pylab inline
import calin.iact_data.raw_actl_event_data_source
import calin.iact_data.telescope_data_source
import json
import struct
import base64

## 1 - Create NectarCam data source

The serialized raw data is included with the __calin__ data structure only if the __include_serialized_raw_data__ option is set in the decoder configuration, as shown below.

In [ ]:
decoder_cfg = calin.iact_data.telescope_data_source.NectarCamZFITSDataSource.default_decoder_config()
decoder_cfg.set_include_serialized_raw_data(True)
src = calin.iact_data.telescope_data_source.NectarCamZFITSDataSource(
    '/CTA/cta.cppm.in2p3.fr/NectarCAM/20161207/Run0268.1.fits.fz', decoder_cfg)

## 2 - Read header and print JSON string

In [ ]:
calin_header = src.get_run_configuration()
print(calin_header.SerializeAsJSON())

## 3 - Extract serialized ACTL header, deserialize it and print JSON string

In [ ]:
cta_serialized_header = calin_header.serialized_raw_header()
cta_header = calin.iact_data.raw_actl_event_data_source.CameraRunHeader()
cta_header.ParseFromString(cta_serialized_header)
cta_str_header = cta_header.SerializeAsJSON();
print(cta_str_header)

## 4 - Read first calin event and print JSON string

In [ ]:
calin_event = src.simple_get_next()
calin_str_event = calin_event.SerializeAsJSON();
print(calin_str_event)

## 5 - Plot waveforms for this event

In [ ]:
high_gain_wfs = calin_event.high_gain_image().camera_waveforms();
for ichan, chan_id in enumerate(high_gain_wfs.channel_id()):
    plot(high_gain_wfs.waveform(ichan).samples())
xlabel('Sample number')
ylabel('Sample amplitude [DC]')

## 6 - Extract serialized ACTL event deserialize it and print JSON string

In [ ]:
cta_serialized_event = calin_event.serialized_raw_event()
cta_event = calin.iact_data.raw_actl_event_data_source.CameraEvent()
cta_event.ParseFromString(cta_serialized_event)
cta_str_event = cta_event.SerializeAsJSON();
print(cta_str_event)

## 7 - Use Python JSON decoder to conver string to Python structure

Print keys of the JSON dictionary

In [ ]:
json_event = json.loads(cta_str_event)
json_event.keys()

## 8 - Convert samples data to integer array

The bytes array from the ACTL Protobuf structure is encoded as a base64 string in the JSON; for details see the mapping guide below:

https://developers.google.com/protocol-buffers/docs/proto3#json

In [ ]:
nsamples = json_event['hiGain']['waveforms']['numSamples']
samples_data = base64.b64decode(json_event['hiGain']['waveforms']['samples']['data'])
samples = frombuffer(samples_data,dtype='int16').reshape([-1,nsamples])
plot(samples.transpose())
xlabel('Sample number')
ylabel('Sample amplitude [DC]')